## **문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템**
### **문제 설명**
*사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.*

### **요구사항**
1. **1단계 체인**: 장르를 입력받아 영화 1편 추천
2. **2단계 체인**: 추천받은 영화의 3줄 줄거리 요약 제공
3. **ChatPromptTemplate** 사용
: “system”  과 “human” 메시지를 지정합니다.
4. 두 체인을 LCEL로 연결    
5. 각 단계의 결과를 모두 출력하여 과정 확인

### **구현 조건**
- 입력: 영화 장르 (예: "액션", "로맨스", "공포")
- 1단계 출력: 추천 영화 제목과 간단한 설명
- 2단계 출력: 영화 줄거리 3줄 요약
### **예상 실행 결과**
입력: "액션"
1단계 결과: "미션 임파서블" 추천
2단계 결과: 
영화: 미션 임파서블
줄거리:
1.특수 요원 이든 헌트가 불가능한 임무를 수행합니다
2.배신과 음모 속에서 진실을 찾아 나갑니다
3.액션과 스릴러가 가득한 스파이 영화입니다


In [58]:
#문제 1-2
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    #api_key=os.getenv("GROQ_API_KEY"), # 실제 API 키 변수명에 맞게 변경하세요.
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)


In [61]:
# Step 1: 사용자가 입력한 장르에 따라 영화 제목 추천
# 시스템 메시지: 영화 제목만 정확히 출력하도록 지시
prompt1 = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영화 평론가입니다. 사용자가 요청하는 장르의 영화 제목을 정확히 1개만 추천해주세요. 다른 설명이나 부가적인 문장 없이 오직 영화 제목만 출력합니다."),
        ("human", "{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")
    ]
)

# Step 2: 추천된 영화의 줄거리 요약
# 시스템 메시지: 3줄 줄거리, 각 줄 앞에 번호와 줄 바꿈 명시
prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영화 줄거리 요약 전문가입니다. 요청받은 영화의 줄거리를 3문장으로 요약해주세요. 각 문장 앞에 번호(1., 2., 3.)를 붙이고, 각 문장 뒤에는 줄 바꿈 문자(\\n)를 포함하여 별도의 줄에 출력해주세요."),
        ("human", "{movie_title} 영화의 줄거리를 3줄로 요약해주세요.")
    ]
)

# Step 3: 최종 답변 형식 지정
# 'system' 메시지에서 '입력: "{genre_input}"' 줄을 완전히 제거합니다.
# 대신 'human' 메시지에서 '입력: "{genre_input}"'을 직접 전달하도록 합니다.
final_output_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 주어진 영화 제목과 줄거리 요약을 다음 형식에 맞춰 출력해야 합니다.
입력: "{genre_input}"
1단계 결과: "{movie_title}" 추천
2단계 결과: 
영화: {movie_title}
줄거리:
{summary_text}
(줄거리는 1. 내용\\n2. 내용\\n3. 내용 형식으로 각 문장이 줄 바꿈되어야 합니다. 추가적인 문장이나 설명 없이 이 양식만 따릅니다.)
"""),
        # 'human' 메시지에 '입력: {genre_input}' 줄을 명확히 추가합니다.
        ("human", "입력: {genre_input}\n1단계 결과: {movie_title} 추천\n2단계 결과:\n영화: {movie_title}\n줄거리:\n{summary_text}\n위 데이터를 바탕으로 최종 양식에 맞춰 출력해 주세요.")
    ]
)

In [62]:
# 체인 1: 영화 제목 추천
movie_title_chain = prompt1 | llm | StrOutputParser()

# 최종 파이프라인
full_pipeline = (
    {
        # RunnableLambda를 사용하여 입력 딕셔너리에서 'genre' 키의 값만 추출합니다.
        "genre_input": RunnableLambda(lambda x: x['genre']),
        
        # 영화 제목을 얻고 'movie_title' 변수로 전달
        "movie_title": movie_title_chain,
        
        # 줄거리 요약 서브 체인을 실행하여 'summary_text' 변수로 전달
        "summary_text": {
            "movie_title": movie_title_chain 
        } | prompt2 | llm | StrOutputParser()
    }
    # 위에서 조합된 {genre_input, movie_title, summary_text} 딕셔너리를
    # final_output_prompt에 전달하여 최종 형식에 맞춥니다.
    | final_output_prompt
    | llm # LLM이 최종 형식에 맞춰 답변 생성
    | StrOutputParser() # 최종 답변을 문자열로 파싱
)

# --- 실행 ---
genre_to_query = "액션" # 사용자 입력 장르

# 전체 파이프라인 실행
final_response = full_pipeline.invoke({"genre": genre_to_query})
print(final_response)

입력: "액션"
1단계 결과: "미션 임파서블" 추천
2단계 결과: 
영화: 미션 임파서블
줄거리: 
1. 이 영화는 국제적인 스파이 조직인 IMF의 에단 헌트(톰 크루즈)가 빌런 어거스트 워커(헨리 카빌)와 대결하는 내용을 담고 있습니다.
2. 영화는 에단 헌트가 자신의 팀원들과 함께 전 세계를 돌아다니며 비밀 작전을 수행하면서 시작됩니다.
3. 에단 헌트는 위험한 상황 속에서도 자신의 목표를 달성하기 위해 노력하며, 영화는 액션과 스파이 활동으로 가득 차 있습니다.
